# Image Preporcessing

There are a few steps for image preprocessing so we can use them for training.
1. Image Extraction: Video recordings of a person will be extracted frame by frame and the frames will be saved as images for subsequent process
2. Face Alignment: The images extrated will go through face detection model to extract the faces with a predefined marginal space
3. Face Encoding: The headshots will be fed into the FaceNet model in order to get the embedding for the face. **Note: this preprocessing might be integrated into training if we are doing fine-tuning where we are adjusting the top layer of the FaceNet model.**

In [1]:
#impoart packages used for visulization in this notebook
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## 1. Image Extraction

In [2]:
# Import packages
import os
from os.path import isdir
from os.path import join

import cv2

**Set the arguments. When running the code as script, one will have to use argparse or similar package to feed to arguments into the script**

In [4]:
args = {
    # Set input directory
    # The names of the subdirectories will be the class labels
    "input_dir":"../face-recognition/data_final/videos",
    # Set output directory
    # The labels will be subdirectories 
    #     where extracted images are saved
    "output_dir":"../face-recognition/data_test2/images",
    "split_types":["train", "val", "test"],
    # Define how many frames to skip before extract each frame 
    "skip":100
}

**Extract frames and save as images**

In [6]:
for split in args["split_types"]:
    input_dir = join(args["input_dir"], split)
    subdirs = [subdir for subdir in os.listdir(input_dir) if isdir(join(input_dir,subdir))]
    # print(subdirs)
    for subdir in subdirs:
        output_subdir = join(args["output_dir"], split, subdir)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)
        for video in os.listdir(join(input_dir,subdir)):
            video_path = join(input_dir,subdir, video)
            vs = cv2.VideoCapture(video_path)
            read = 0
            saved = 0
            # loop over frames from the video file stream
            while True:
                # grab the frame from the file
                (grabbed, frame) = vs.read()
                # if the frame was not grabbed, then we have reached the end
                # of the stream
                if not grabbed:
                    break
                # increment the total number of frames read thus far
                read += 1
                # check to see if we should process this frame
                if read % args["skip"] != 0:
                    continue
                output_img_path = join(output_subdir,"{}.png".format(saved))
                cv2.imwrite(output_img_path, frame)
                saved += 1
                print("[INFO] saved {} to disk".format(output_img_path))

[INFO] saved ../face-recognition/data_test2/images/train/erik/0.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/1.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/2.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/3.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/4.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/5.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/6.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/7.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/8.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/9.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/10.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/11.png to disk
[INFO] saved ../face-recognition/data_test2/images/train/erik/12.png to disk
[INFO] sa

## Face Alignment

Once we have a bunch of images, we can align the faces in those images and extract them, and again save them as images.

In [8]:
# Import packages
import os
from os.path import isdir
from os.path import join
import importlib

import numpy as np
import cv2

from src import detectors
# from common_resources.detectors import FaceDetector


importlib.reload(detectors)
FaceDetector = detectors.FaceDetector

We have a few options for face detections:
1. The detector in HW7
2. Open CV detector

Build a wrapper so we can use them in the same way

In [6]:
args = {
    # Set input directory
    # The names of the subdirectories will be the class labels
    "input_dir":"data/images/",
    # Set output directory
    # The labels will be subdirectories 
    #     where extracted faces are saved
    "output_dir":"data/faces_hw7/",
    # Define the margin (extra space) for face alignment
    "margin":10,
    "split_types":["train", "val", "test"],
    # Define the face detector used and paramters
    # for OpenCV ("OPENCV", MODEL_FILE_PATH, scaleFactor, MinNeighbors)
    # for HW7 ("HW7", MODEL_FILE_PATH, DETECTION_THRESHOLD)
#     "detector": ("OPENCV", 
#                  "src/detector/haarcascade_frontalface_default.xml", 
#                  1.3, 
#                  5)
#     "detector": ("OPENCV", 
#                  "src/detector/haarcascade_frontalface_alt2.xml", 
#                  1.1, 
#                  3)
    "detector": ("HW7", 
                 "src/detectors/frozen_inference_graph_face.pb", 
                 0.7)
#     "detector": ("DLIB",)    
}

In [7]:
# dlib_face_detector = FaceDetector(args["detector"])
# hw7_face_detector = FaceDetector(("HW7", "detector/frozen_inference_graph_face.pb", 0.5))
# opencv_face_detector = FaceDetector(("OPENCV", "detector/haarcascade_frontalface_default.xml", 1.3, 5))

In [8]:
# for i in range(20):
#     # get sample images
#     image = cv2.imread(args["input_dir"] + f"/diana/{i}.png")
#     print(args["input_dir"])
#     fig = plt.figure()
#     ax = fig.add_subplot(1, 1, 1)

#     ax.imshow(image)
#     x,y,w,h = hw7_face_detector.detect_faces(image)[0]
#     print(x,y,w,h, w*h)
#     x,y,w,h = opencv_face_detector.detect_faces(image)[0]
#     print(x,y,w,h, w*h)
#     x,y,w,h = face_detector.detect_faces(image)[0]
#     print(x,y,w,h, w*h)
#     # display rectangle
#     patch = patches.Rectangle((x, y), w, h, color='g', alpha=0.3)
#     ax.add_patch(patch)

#     plt.show()

In [9]:
# Initialize the detector of choice
face_detector = FaceDetector(args["detector"])
margin = args["margin"]

for split in args["split_types"]:
    input_dir = join(args["input_dir"], split)
    subdirs = [subdir for subdir in os.listdir(input_dir) if isdir(join(input_dir,subdir))]
    
    for subdir in subdirs:
        output_subdir = join(args["output_dir"], split, subdir)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)
        saved = 0
        for file in os.listdir(join(input_dir, subdir)):
            image_path = join(input_dir, subdir, file)
            image = cv2.imread(image_path)

            faces = face_detector.detect_faces(image)
            for (x,y,w,h) in faces:
                x_start = max(0,x-margin//2)
                y_start = max(0,y-margin//2)
                x_end = min(image.shape[1], x+w+margin//2)
                y_end = min(image.shape[0], y+h+margin//2)
                cropped = image[y_start:y_end, x_start:x_end]
                # resize the image to 160x160 for FaceNet
                aligned = cv2.resize(cropped, (160, 160))
                output_img_path = join(output_subdir,"face_{}.png".format(saved))
                cv2.imwrite(output_img_path, aligned)
                saved += 1
                print("[INFO] saved {} to disk".format(output_img_path))

[INFO] saved data/faces_hw7/train/erik/face_0.png to disk
[INFO] saved data/faces_hw7/train/erik/face_1.png to disk
[INFO] saved data/faces_hw7/train/erik/face_2.png to disk
[INFO] saved data/faces_hw7/train/erik/face_3.png to disk
[INFO] saved data/faces_hw7/train/erik/face_4.png to disk
[INFO] saved data/faces_hw7/train/erik/face_5.png to disk
[INFO] saved data/faces_hw7/train/erik/face_6.png to disk
[INFO] saved data/faces_hw7/train/erik/face_7.png to disk
[INFO] saved data/faces_hw7/train/erik/face_8.png to disk
[INFO] saved data/faces_hw7/train/erik/face_9.png to disk
[INFO] saved data/faces_hw7/train/erik/face_10.png to disk
[INFO] saved data/faces_hw7/train/erik/face_11.png to disk
[INFO] saved data/faces_hw7/train/erik/face_12.png to disk
[INFO] saved data/faces_hw7/train/erik/face_13.png to disk
[INFO] saved data/faces_hw7/train/erik/face_14.png to disk
[INFO] saved data/faces_hw7/train/erik/face_15.png to disk
[INFO] saved data/faces_hw7/train/erik/face_16.png to disk
[INFO] 

[INFO] saved data/faces_hw7/train/diana/face_67.png to disk
[INFO] saved data/faces_hw7/train/diana/face_68.png to disk
[INFO] saved data/faces_hw7/train/diana/face_69.png to disk
[INFO] saved data/faces_hw7/train/diana/face_70.png to disk
[INFO] saved data/faces_hw7/train/diana/face_71.png to disk
[INFO] saved data/faces_hw7/train/diana/face_72.png to disk
[INFO] saved data/faces_hw7/train/diana/face_73.png to disk
[INFO] saved data/faces_hw7/train/diana/face_74.png to disk
[INFO] saved data/faces_hw7/train/diana/face_75.png to disk
[INFO] saved data/faces_hw7/train/diana/face_76.png to disk
[INFO] saved data/faces_hw7/train/diana/face_77.png to disk
[INFO] saved data/faces_hw7/train/diana/face_78.png to disk
[INFO] saved data/faces_hw7/train/diana/face_79.png to disk
[INFO] saved data/faces_hw7/train/diana/face_80.png to disk
[INFO] saved data/faces_hw7/train/diana/face_81.png to disk
[INFO] saved data/faces_hw7/train/diana/face_82.png to disk
[INFO] saved data/faces_hw7/train/diana/

[INFO] saved data/faces_hw7/train/unknown/face_113.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_114.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_115.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_116.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_117.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_118.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_119.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_120.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_121.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_122.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_123.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_124.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_125.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_126.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_127.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_128.png 

[INFO] saved data/faces_hw7/train/unknown/face_245.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_246.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_247.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_248.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_249.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_250.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_251.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_252.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_253.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_254.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_255.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_256.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_257.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_258.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_259.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_260.png 

[INFO] saved data/faces_hw7/train/unknown/face_377.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_378.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_379.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_380.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_381.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_382.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_383.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_384.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_385.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_386.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_387.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_388.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_389.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_390.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_391.png to disk
[INFO] saved data/faces_hw7/train/unknown/face_392.png 

[INFO] saved data/faces_hw7/val/diana/face_36.png to disk
[INFO] saved data/faces_hw7/val/diana/face_37.png to disk
[INFO] saved data/faces_hw7/val/diana/face_38.png to disk
[INFO] saved data/faces_hw7/val/diana/face_39.png to disk
[INFO] saved data/faces_hw7/val/diana/face_40.png to disk
[INFO] saved data/faces_hw7/val/diana/face_41.png to disk
[INFO] saved data/faces_hw7/val/diana/face_42.png to disk
[INFO] saved data/faces_hw7/val/diana/face_43.png to disk
[INFO] saved data/faces_hw7/val/diana/face_44.png to disk
[INFO] saved data/faces_hw7/val/diana/face_45.png to disk
[INFO] saved data/faces_hw7/val/diana/face_46.png to disk
[INFO] saved data/faces_hw7/val/diana/face_47.png to disk
[INFO] saved data/faces_hw7/val/diana/face_48.png to disk
[INFO] saved data/faces_hw7/val/diana/face_49.png to disk
[INFO] saved data/faces_hw7/val/diana/face_50.png to disk
[INFO] saved data/faces_hw7/val/diana/face_51.png to disk
[INFO] saved data/faces_hw7/val/diana/face_52.png to disk
[INFO] saved d

[INFO] saved data/faces_hw7/val/unknown/face_84.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_85.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_86.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_87.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_88.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_89.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_90.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_91.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_92.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_93.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_94.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_95.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_96.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_97.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_98.png to disk
[INFO] saved data/faces_hw7/val/unknown/face_99.png to disk
[INFO] saved data/faces_hw7/val/unknown/

[INFO] saved data/faces_hw7/test/unknown/face_14.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_15.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_16.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_17.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_18.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_19.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_20.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_21.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_22.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_23.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_24.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_25.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_26.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_27.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_28.png to disk
[INFO] saved data/faces_hw7/test/unknown/face_29.png to disk
[INFO] saved data/faces_

[INFO] saved data/faces_hw7/test/unknown/face_149.png to disk


## Face Encoding

Now that we have the faces extracted and aligned, we could encode the faces with the FaceNet model for training the SVM. Note: this step is not necessary for fine-tuning on a neural classifier

In [1]:
# Import packages
import os
from os.path import isdir
from os.path import join
import importlib

import numpy as np
import cv2

from src import face_encoders

importlib.reload(face_encoders)
FaceEncoder = face_encoders.FaceEncoder

Using TensorFlow backend.


In [2]:
encoder_model = FaceEncoder("facenet_keras", "src/encoders/facenet_keras.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
args = {
    # Set input directory
    # The names of the subdirectories will be the class labels
    "input_dir":"data/faces_dlib/",
    # Set output directory
    # The labels will be subdirectories 
    #     where extracted faces are saved
    "output_dir":"data/embeddings_dlib/",
    "split_types":["train", "val", "test"],
}

In [7]:
if not os.path.exists(args["output_dir"]):
    os.makedirs(args["output_dir"])

for split in args["split_types"]:
    print(f"[INFO] Encoding faces in {split} set...")
    input_dir = join(args["input_dir"], split)
    subdirs = [subdir for subdir in os.listdir(input_dir) if isdir(join(input_dir,subdir))]
    
    faces = list()
    embeddings = list()
    labels = list()
    for label in subdirs:
        files = os.listdir(join(input_dir, label))
        print(f'    [INFO] Encoding {len(files)} "{label}" faces...')
        for file in files:
            face_path = join(input_dir, label, file)
            face = cv2.imread(face_path)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face_array = np.asarray(face)
            embedding = encoder_model.get_embedding(face_array)
            faces.append(face_array)
            embeddings.append(embedding)
            labels.append(label)
    faces = np.asarray(faces)
    embeddings = np.asarray(embeddings)
    labels = np.asarray(labels)
    output_face_file = join(args["output_dir"], f"{split}_faces.npz")
    output_emb_file = join(args["output_dir"], f"{split}_embeddings.npz")
    np.savez_compressed(output_face_file, faces, labels)
    np.savez_compressed(output_emb_file, embeddings, labels)
    print(f"[INFO] Saved {split} set as {output_face_file}, shape: {str(embeddings.shape)}, {str(labels.shape)}")

[INFO] Encoding faces in train set...
    [INFO] Encoding 139 "erik" faces...
    [INFO] Encoding 97 "diana" faces...
    [INFO] Encoding 479 "unknown" faces...
[INFO] Saved train set as data/embeddings_dlib/train_faces.npz, shape: (715, 128), (715,)
[INFO] Encoding faces in val set...
    [INFO] Encoding 23 "erik" faces...
    [INFO] Encoding 98 "diana" faces...
    [INFO] Encoding 158 "unknown" faces...
[INFO] Saved val set as data/embeddings_dlib/val_faces.npz, shape: (279, 128), (279,)
[INFO] Encoding faces in test set...
    [INFO] Encoding 30 "erik" faces...
    [INFO] Encoding 33 "diana" faces...
    [INFO] Encoding 156 "unknown" faces...
[INFO] Saved test set as data/embeddings_dlib/test_faces.npz, shape: (219, 128), (219,)
